# ML Pipeline 
按照如下的指导要求，搭建你的机器学习管道。
### 1. 导入与加载
- 导入 Python 库
- 使用 [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html) 从数据库中加载数据集
- 定义特征变量X 和目标变量 Y

In [ ]:
    category_names = df.iloc[:,4:].columns
    category_boolean = (df.iloc[:,4:] != 0).sum().sort_values(ascending = False).values
      genre_counts = df.groupby('genre').count()['message']
    genre_names = list(genre_counts.index)
    
    

In [ ]:
    graphs = [
        {
            'data': [
                Bar(
                    x=genre_names,
                    y=genre_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Message Genres',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Genre"
                }
            }
        },
            # GRAPH 2 - category graph    
        {
            'data': [
                Bar(
                    x=category_names,
                    y=category_boolean
                )
            ],

            'layout': {
                'title': 'Distribution of Message Categories',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Category",
                    'tickangle': 35
                }
            }
            
            #GRAPH 3 - genre
        }
    ]

In [13]:
grouped_genre = df.groupby('genre').sum()
grouped_genre_value = grouped_genre.iloc[:,1:].values
grouped_genre_index = grouped_genre.index

In [14]:
grouped_genre_value = grouped_genre.iloc[:,1:].values
grouped_genre_index = grouped_genre.index

In [16]:
grouped_genre_index

Index(['direct', 'news', 'social'], dtype='object', name='genre')

In [2]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy
import sqlite3
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords','averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("InsertTableName", con = engine)
#X = 
#Y = 

In [5]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [6]:
df = df.dropna(axis = 0)

In [7]:
X = df['message']
Y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]

### 2. 编写分词函数，开始处理文本

In [8]:
import re
marker = re.compile(r'[^a-zA-Z]')
X = X.replace(marker, " ")

In [9]:
from nltk.corpus import stopwords

In [71]:
filtered_words = [word for word in list(X[1]) if word not in stopwords.words('english')]
print('原始词：', X[1])
print('去除停用词后：', filtered_words)

原始词： Is the Hurricane over or is it not over
去除停用词后： ['I', ' ', 'h', 'e', ' ', 'H', 'u', 'r', 'r', 'c', 'n', 'e', ' ', 'v', 'e', 'r', ' ', 'r', ' ', ' ', ' ', 'n', ' ', 'v', 'e', 'r']


In [10]:
def tokenize2(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    
    cleaned_tokens = []
    for tok in tokens:
        cleaned_tok = lemmatizer.lemmatize(tok).lower().strip()
        
        cleaned_tokens.append(cleaned_tok)
        
    cleaned_tokens = [word for word in cleaned_tokens if word not in stopwords.words('english')]
    
    return cleaned_tokens

In [11]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    #tokens_without_stopwords = [word for word in tokens  if word not in stopwords.words('english')]
    
    cleaned_tokens = []
    for tok in tokens:
        cleaned_tok = lemmatizer.lemmatize(tok).lower().strip()
        
        cleaned_tokens.append(cleaned_tok)
    
    return cleaned_tokens

In [12]:
from sklearn.model_selection import train_test_split
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()

In [17]:
clf = MultiOutputClassifier(RandomForestClassifier(random_state=42))

In [13]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42, test_size = 0.3)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42, test_size = 0.3)
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)

In [15]:
clf.fit(X_train_tfidf, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1)

In [22]:
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
#y_pred = clf.predict(X_test_tfidf)

In [56]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [129]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

        0.0       0.34      0.11      0.17      1861
        1.0       0.77      0.93      0.84      5951
        2.0       0.00      0.00      0.00        51

avg / total       0.66      0.73      0.68      7863

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

        0.0       0.84      0.98      0.91      6555
        1.0       0.44      0.08      0.14      1308

avg / total       0.77      0.83      0.78      7863

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7831
        1.0       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      7863

------------------------------------------------------

FEATURE: aid_related

    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 3. 创建机器学习管道 
这个机器学习管道应该接收 `message` 列作输入，输出分类结果，分类结果属于该数据集中的 36 个类。你会发现 [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) 在预测多目标变量时很有用。

In [15]:
from sklearn.pipeline import Pipeline
from nltk import pos_tag


In [194]:
pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize2)),
    ("tfidf", TfidfTransformer()),
    ('Randomforest', MultiOutputClassifier(RandomForestClassifier())),
])

In [195]:
y_pred.dtype

dtype('float64')

In [196]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

In [93]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

        0.0       0.31      0.13      0.19      1861
        1.0       0.76      0.91      0.83      5951
        2.0       0.17      0.02      0.04        51

avg / total       0.65      0.72      0.67      7863

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

        0.0       0.85      0.96      0.90      6555
        1.0       0.39      0.12      0.19      1308

avg / total       0.77      0.82      0.78      7863

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7831
        1.0       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      7863

------------------------------------------------------

FEATURE: aid_related

    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [186]:
vect = CountVectorizer(tokenizer=tokenize2)
tfidf = TfidfTransformer()

In [35]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(new_X, Y, random_state = 42, test_size = 0.3)
X_train_counts_new = vect.fit_transform(X_train_new)
X_train_tfidf_new = tfidf.fit_transform(X_train_counts_new)

In [52]:
from sklearn.pipeline import FeatureUnion

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [187]:
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)

In [189]:
X_train_2 = TfidfVectorizer(tokenizer=tokenize2).fit_transform(X_train)

In [205]:
X_train.shape

(7156,)

In [206]:
sn = starting_verb_classifier()
K = sn.transform(X_train)

In [214]:
K.isnull().sum()

message    3
dtype: int64

In [ ]:
TfidfVectorizer(tokenizer = tokenize2).fit_transform(X_train)

In [53]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([
        
        ('tfidf_vect', TfidfVectorizer(tokenizer = tokenize2)),  
        ('starting_noun', starting_noun_classifier()),
        ('noun_word_ratio', NounWordRatio())
        
    ])),
    ('randomforest', MultiOutputClassifier(RandomForestClassifier()))
])

In [54]:
pipeline2.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tfidf_vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ng...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [57]:

y_pred2 = pipeline2.predict(X_test)
y_pred_pd2 = pd.DataFrame(y_pred2, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd2[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

        0.0       0.38      0.20      0.27      1048
        1.0       0.65      0.82      0.73      1977
        2.0       0.17      0.02      0.04        42

avg / total       0.55      0.60      0.56      3067

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

        0.0       0.66      0.89      0.76      1990
        1.0       0.40      0.14      0.21      1077

avg / total       0.57      0.63      0.56      3067

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      3063
        1.0       0.00      0.00      0.00         4

avg / total       1.00      1.00      1.00      3067

------------------------------------------------------

FEATURE: aid_related

    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
sentence_list = nltk.sent_tokenize(X[0])
sentence_list

['Weather update   a cold front from Cuba that could pass over Haiti']

In [34]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [39]:
word = nltk.word_tokenize(X[0])
pos_tags = nltk.pos_tag(word)

In [44]:
noun = 0
for i in pos_tags:
    if i[1] == 'NNP':
        noun += 1

In [46]:
noun / len(word)

0.25

In [ ]:
word = nltk.word_tokenize(text)
pos_tags = nltk.pos_tag(word)
noun_length = 0
for value in pos_tags:
    if value[1] in ['NN', 'NNP','NNS','NNPS']:
        noun_length += 1
return noun_length / len(word)

In [47]:
from sklearn.base import BaseEstimator, TransformerMixin
class NounWordRatio(BaseEstimator, TransformerMixin):

    def noun_ratio(self, text):
        # tokenize by word
        word = nltk.word_tokenize(text)
        pos_tags = nltk.pos_tag(word)
        noun_length = 0
        #calculate the number of noun word
        for value in pos_tags:
            if value[1] in ['NN', 'NNP','NNS','NNPS']:
                noun_length += 1
                
        if len(word) > 0:
            return noun_length / len(word)
        else:
            return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_Cal = pd.Series(X).apply(self.noun_ratio)

        df_X_Cal = pd.DataFrame(X_Cal)
        return df_X_Cal.fillna(0)

In [50]:
from sklearn.base import BaseEstimator, TransformerMixin
class starting_noun_classifier(BaseEstimator, TransformerMixin):

    def starting_noun(self, text):
        # tokenize by sentences
        
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['NN', 'NNP','NNS','NNPS']:
                return 1

            return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_noun)

        df_X_tagged = pd.DataFrame(X_tagged)
        return df_X_tagged.fillna(0)

In [ ]:
def evaluate_model(model, X_test, Y_test, category_names):
    
    y_pred = model.predict(X_test)
    df_pred = pd.DataFrame(y_pred, columns = category_names)
    for column in category_names:
        print('------------------------------------------------------\n')
        print('Feature_Name: {}\n'.format(column))
        print(classification_report(Y_test[column],df_pred[column]))

evaluate_model(model, X_test, Y_test, category_names)

In [ ]:
with open('model.pickle', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
import pickle
s=pickle.dumps(clf)
f=open('svm.txt','w')
f.write(s)
f.close()
f2=open('svm.txt','r')
s2=f2.read()
clf2=pickle.loads(s2)
clf2.score(X,y)

### 4. 训练管道
- 将数据分割成训练和测试集
- 训练管道

### 5. 测试模型
报告数据集中每个输出类别的 f1 得分、准确度和召回率。你可以对列进行遍历，并对每个元素调用 sklearn 的 `classification_report`。

### 6. 优化模型
使用网格搜索来找到最优的参数组合。 

In [62]:
     pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize2)),
                ('tfidf', TfidfTransformer())
            ])),

            ('NounWordRatio', NounWordRatio())
        ])),
    
        ('RandomForest', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [63]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [64]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_...rm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('NounWordRatio', NounWordRatio())],
          transformer_weights=None)),
  ('RandomForest',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
            

In [66]:
from sklearn.grid_search import GridSearchCV

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [81]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'RandomForest', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__NounWordRatio', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'feature

In [82]:
parameters = {
        'RandomForest__estimator__n_estimators': [50, 100, 200],
        'RandomForest__estimator__min_samples_split': [2, 3],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'NounWordRatio': 0.5},
            {'text_pipeline': 0.5, 'NounWordRatio': 1}
        )
    }

cv = GridSearchCV(pipeline, param_grid= parameters)

### 7. 测试模型
打印微调后的模型的精确度、准确率和召回率。  

因为本项目主要关注代码质量、开发流程和管道技术，所有没有模型性能指标的最低要求。但是，微调模型提高精确度、准确率和召回率可以让你的项目脱颖而出——特别是让你的简历更出彩。

In [88]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            
            ('NounWordRatio', NounWordRatio())
        ])),    
        ('RandomForest', MultiOutputClassifier(RandomForestClassifier()))
        ])
    
    return pipeline



"""Return the ratio of noun words"""

### 8. 继续优化模型，比如：
* 尝试其他的机器学习算法
* 尝试除 TF-IDF 外其他的特征

In [ ]:
def build_model():
     pipeline = Pipeline([
    ('features', FeatureUnion([
            
        ('text_pipeline', Pipeline([
                 ('vect', CountVectorizer(tokenizer=tokenize)),
                 ('tfidf', TfidfTransformer())
            ])),
            
    ('NounWordRatio', NounWordRatio())
        ])),    
    ('RandomForest', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline

### 9. 导出模型为 pickle file

### 10. Use this notebook to complete `train.py`
使用资源 (Resources)文件里附带的模板文件编写脚本，运行上述步骤，创建一个数据库，并基于用户指定的新数据集输出一个模型。